In [4]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma import aims
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
a = ana.Anatomist()

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module volumepalettes
loading module paletteViewer
loading module modelGraphs
loading module infowindow
loading module profilewindow
loading module bsa_proba
loading module meshsplit
loading module selection
loading module anacontrolmenu
loading module gltf_io
loading module ana_image_math
loading module palettecontrols
loading module statsplotwindow
loading module foldsplit
loading module histogram
loading module save_resampled
loading module simple_controls
loading module valuesplotwindow
loading module gradientpalette
all python modules loaded
Anatomist started.


### Variable definitions

In [12]:
#embeddings_file = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/ScCal-SLi_right/09-28-12_1/imagen_random_embeddings/full_embeddings.csv"
#embeddings_file = "/neurospin/dico/jlaval/Output/imagen_right/22-26-14_254/imagen_random_embeddings/full_embeddings.csv"
#participants_file = "/neurospin/dico/data/deep_folding/current/datasets/imagen/patterns/interruptions.csv"
#participants_file = "/neurospin/dico/data/deep_folding/current/datasets/imagen/participants.csv"
embeddings_file = "/neurospin/dico/jlaval/Output/5_models_FIP_right_3_layer_proj/pepper_05_0/FIP_right_custom_embeddings/custom_cross_val_embeddings.csv"
participants_file = "/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/FIP_labels.csv"

In [13]:
#label = "C0_L"
#label = "Right_Interrup_RS_OTS"
#label = "Right_Interrup_RS_CS"
#label = "Right_Interrup_CS_OTS"
label = 'Right_FIP'

In [33]:
side = "R" # "R" or "L"
#region = "Sc.Cal.-S.Li."# "S.C.-sylv.", "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."
#region = "deMatos."
#database='imagen'
database='hcp'
region = 'F.I.P.'

# Building predictors

In [16]:
participants = pd.read_csv(participants_file, dtype={'Subject':object})
participants = participants.set_index("Subject")
participants = participants[[label]]

In [17]:
participants.head()

,Right_FIP
Subject,
100206,1
100307,1
100610,1
101309,1
101410,0


In [18]:
participants = participants[[label]].dropna()
participants = participants.replace({"N": 0, "?": 1, "Y": 2})
participants[label].unique()

array([1, 0])

In [19]:
len(participants)

390

In [20]:
embeddings = pd.read_csv(embeddings_file, dtype={'ID':object})
embeddings = embeddings.set_index("ID")

In [21]:
embeddings

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30,dim31,dim32
ID,,,,,,,,,,,,,,,,,,,,,
101309,-4.645955,0.768577,-4.262807,-1.701820,1.482541,-8.389413,-4.684514,4.029183,-0.317528,-1.284137,...,0.770991,-6.611512,5.936325,-5.235019,2.263759,-1.482382,-10.483747,2.803975,5.981444,2.832390
101410,3.018876,6.386819,-1.604598,-4.527468,-7.011570,10.830564,-0.153391,-11.147780,1.775307,5.354484,...,-1.788843,-2.710631,-6.246329,8.278157,0.179346,-1.099906,1.535612,-3.286289,2.629743,0.222221
102311,-1.703207,5.305471,3.311594,-6.217149,-5.283031,-3.715484,-2.444225,-7.062653,4.978241,0.181690,...,-11.711569,0.523505,5.763735,-4.444607,-0.916646,-1.255991,-4.318297,-4.389846,-3.419158,-5.535358
102513,-1.218514,7.885838,-0.246438,-1.105210,-0.408006,6.814372,3.458224,-6.853157,0.915468,3.235942,...,2.327060,-4.878108,0.621870,6.417227,1.818214,-1.548059,0.291248,2.519700,-0.451415,6.325996
102715,-2.438971,8.448846,0.194177,-2.697090,-6.852919,4.188613,1.251174,-12.089166,5.689527,5.060450,...,2.313338,-3.718028,-7.456728,5.030943,-0.591240,-5.062529,-3.469988,2.840681,1.907922,3.357096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983773,2.804344,-3.708754,-0.487654,0.291813,-1.829341,-7.400002,-7.045345,6.743644,1.370303,-2.605541,...,-6.577653,-0.128884,3.506489,-9.045783,-1.377032,-0.630945,-7.776711,-0.868080,7.206292,-3.808692
984472,1.362595,-2.989746,-0.898230,5.481171,0.968816,-3.369124,-5.830699,4.283732,-0.824093,1.832710,...,-3.989450,2.628019,-1.364065,0.244887,9.082562,-0.389757,-0.876966,1.303839,6.169351,3.646609
991267,4.440444,-8.213302,9.098431,-4.059246,-3.640297,-4.382610,-1.519791,2.362434,5.334361,1.002724,...,-7.998389,7.492885,0.930221,-11.493196,-4.055149,-4.144620,0.446769,-1.828778,0.091363,-5.929795


In [22]:
merged = participants[[label]].merge(embeddings, left_index=True, right_index=True)

# Classifies
X = merged.drop([label], axis=1)
Y = merged[[label]].astype(float)

# Standard scaler
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)
df2 = X.copy()

# Makes OLS
df2 = sm.add_constant(df2)
model = sm.OLS(Y[label], df2)
results = model.fit()

p = results.params

# Gets predictor
df = Y.copy()
df["predicted"] = df2.dot(p)

results.fvalue

7.222043668105118

In [25]:
## here use directly the classif scores
#dir_proba = f'/neurospin/dico/jlaval/Output/imagen_right/22-26-14_254/imagen_random_embeddings/{label}/full_predicted_probas.csv'
dir_proba = '/neurospin/dico/jlaval/Output/5_models_FIP_right_3_layer_proj/pepper_05_0/FIP_right_custom_embeddings/Right_FIP/test_predicted_probas.csv'
df = pd.read_csv(dir_proba)
df_ = pd.DataFrame({label: df['label'], 'predicted': df['mean_pred']})
df_.index=df['Subject']
df = df_.copy()

In [26]:
df.mean()

Right_FIP    0.692308
predicted    0.696379
dtype: float64

In [27]:
df.head()

,Right_FIP,predicted
Subject,,
101309,1,0.887186
101410,0,0.851128
102311,1,0.947392
102513,0,0.592449
102715,1,0.964249


### Function definitions

In [34]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() == 'imagen':
            subject_id_list = [str(num).zfill(12) for num in subject_id_list]
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        elif dataset_name_list[rep].lower() in ['imagen']:
            dataset = 'imagen'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path.isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [39]:
def visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=200, nb_columns=2, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    try:
        _block
    except NameError:
        _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name)
    print(df[-30:])
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
        
    # Ensures that last list contains the last step subjects
    list_idx = (predict_proba.index[-step:].to_numpy())
    dic_packages[i//step] = list_idx
        
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    list_pack[-1] = n_pack-1
    for i in list_pack:
        sum_vol = buckets_average(dic_packages[i], list_database)
        _average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol)
        _average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
        #wsum = a.createWindow('Sagittal', block=block)
        #wsum.addObjects(a_sum_vol)
        _average_dic[f'rvol{i}'] = a.fusionObjects(
            objects=[_average_dic[f'a_sum_vol{i}']],
            method='VolumeRenderingFusionMethod')
        _average_dic[f'rvol{i}'].releaseAppRef()
        # custom palette
        n = len(dic_packages[i])
        pal = a.createPalette('VR-palette')
        pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
        build_gradient(pal)
        _average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
        pal2 = a.createPalette('slice-palette')
        pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
        build_gradient(pal2)
        _average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
        # rvol.palette().fill()
        _average_dic[f'wvr{i}'] = a.createWindow('3D', block=_block)
        _average_dic[f'wvr{i}'].addObjects(_average_dic[f'rvol{i}'])

### Visualization

In [40]:
# block = a.createWindowsBlock(10)
visualize_averages_along_parameter(df, "predicted", database)

         Right_FIP  predicted
Subject                      
761957           1   0.981754
800941           1   0.982570
104012           1   0.983265
206525           1   0.983881
204319           1   0.984121
213421           1   0.986293
281135           1   0.986340
926862           1   0.986765
196346           1   0.988921
448347           1   0.989259
139839           1   0.989951
134324           1   0.990065
814649           1   0.990110
677766           1   0.990163
679770           1   0.990340
378756           1   0.990404
144933           1   0.990452
187143           1   0.990694
119833           1   0.990854
144731           1   0.991367
983773           1   0.992679
623844           1   0.993217
843151           1   0.993280
219231           1   0.993359
419239           1   0.993801
991267           1   0.995355
167743           1   0.995777
191942           1   0.995826
852455           1   0.996260
774663           1   0.997836
<soma.aims.Volume_DOUBLE object at 0x7aa

In [137]:
visualize_averages_along_parameter(df, label, database)

          Right_Interrup_RS_OTS  predicted
Subject                                   
42396259                      1   0.131393
42288666                      1   0.181225
41434646                      1   0.236215
41177354                      1   0.162954
40983467                      1   0.045086
40590177                      1   0.159340
40485442                      1   0.173944
54389991                      1   0.170944
58846772                      1   0.040276
58754043                      1   0.474252
57968987                      1   0.085409
57944398                      1   0.213228
57541508                      1   0.413152
57515374                      1   0.060265
56864858                      1   0.673406
55478711                      1   0.296330
54788610                      1   0.136534
54741718                      1   0.392204
54554147                      1   0.150667
88106971                      1   0.129852
53790569                      1   0.047158
53730509   

In [18]:
type(_block)

anatomist.direct.api.Anatomist.AWindowsBlock

In [19]:
_average_dic

{'a_sum_vol0': <anatomist.cpp.anatomist.SliceableObject object at 0x7fcab0915ea0>,
 'rvol0': <anatomist.cpp.anatomist.MObject object at 0x7fca85764940>,
 'wvr0': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7fca74394f70>,
 'a_sum_vol4': <anatomist.cpp.anatomist.SliceableObject object at 0x7fcab0993c70>,
 'rvol4': <anatomist.cpp.anatomist.MObject object at 0x7fca74362050>,
 'wvr4': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7fca74362290>,
 'a_sum_vol10': <anatomist.cpp.anatomist.SliceableObject object at 0x7fca743620e0>,
 'rvol10': <anatomist.cpp.anatomist.MObject object at 0x7fca743b0a60>,
 'wvr10': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7fca743b11b0>}

QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout
